In [96]:
import aspect_based_sentiment_analysis as absa
import json
import pandas as pd
from textblob import TextBlob
import re

In [97]:
with open("C:/Users/Chitra Bhat/Desktop/tweets.json") as jfile:
    d = json.load(jfile)

In [98]:
data = []
for keys in d.keys():
    data.append(d[keys])

In [99]:
df = pd.DataFrame.from_dict(data)

In [100]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).lower().split())

In [101]:
df['tweet_text'] = df['tweet_text'].apply(clean_tweet)

In [73]:
list_of_entities = []
for tweet in df['tweet_text']:
    blob = TextBlob(tweet)
    list_of_entities.extend(blob.noun_phrases)

In [86]:
from collections import Counter
c = Counter(list_of_entities)

In [1]:
objective1 = pd.DataFrame.from_dict(c.items())
objective1 = objective1.rename(columns = {0: 'entity', 1: 'frequency'})
objective1 = objective1.sort_values(by=['frequency'], ascending = False)
objective1.reset_index(inplace = True)

In [27]:
df=df.set_index('tweet_author')

In [311]:
nlp = absa.load()
dataframe = pd.DataFrame(columns = ["Entity", "Sentiment", "Name"])
for text in list(df['tweet_text']):
   # tinydf = pd.DataFrame()
    #print(text)
    sentiment_list = {}
    noun_phrases = list(TextBlob(text).noun_phrases)
    #print(noun_phrases)
    for noun in noun_phrases:
        word = list(nlp(text, aspects = [noun]).subtasks.values())[0].sentiment
        if word == absa.Sentiment.positive:
            sentiment_list[noun] = 'Positive'
        elif word == absa.Sentiment.negative:
            sentiment_list[noun] = 'Negative'
        else: sentiment_list[noun] = 'Neutral'
    tinydf = pd.DataFrame(sentiment_list.items(), columns = ["Entity", "Sentiment"]) 
    tinydf["Name"] = list(df.loc[df['tweet_text'] == text, 'tweet_author'])[0]
    dataframe = dataframe.append(tinydf)

    

In [305]:
sorted_df = dataframe.sort_values(by='Entity', ascending=True)

In [306]:
sorted_df = sorted_df.reset_index()
cols = ['Entity', 'Name', 'Sentiment']
sorted_df = sorted_df[cols]

In [309]:
sorted_df.rename(columns = {'Entity':'entity', 'Name': 'author_name', 'Sentiment':'overall_polarity'}, inplace = True)
sorted_df.to_csv('objective2.csv', index = False)